In [ ]:
pip install finance-datareader

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

%matplotlib inline
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumGothic'

In [ ]:
import FinanceDataReader as fdr
df_krx = fdr.StockListing('KRX')
df_krx.head()

In [ ]:
hdcar = fdr.DataReader('005380')# 현대차 종목번호 = 005380

In [ ]:
hdcar

# 데이터 설명
- Open : 시초가
- High : 고가
- Low : 저가
- Close : 종가
- Volumne : 거래량
- Change : 대비

In [ ]:
hdcar.tail()

In [ ]:
# 특정 연도 기준 이후의 데이터 필요시
hdcar_2022 = fdr.DataReader('005380', '2022')
hdcar_2022

In [ ]:
# 최근 20년간 삼성전자 주가 모음
hdcar_20 = fdr.DataReader('005380', '2002')
hdcar_20

In [ ]:
hdcar_20.describe

In [ ]:
hdcar_20.index

In [ ]:
# DatetimeIndex = 연도/월/일 split 가능
hdcar_20['Year'] = hdcar_20.index.year
hdcar_20['Month'] = hdcar_20.index.month
hdcar_20['Day'] = hdcar_20.index.day

In [ ]:
hdcar_20

In [ ]:
import matplotlib as mpl
%matplotlib inline

plt.figure(figsize = (20, 12))
plt.plot(hdcar_20.index, hdcar_20['Close'])# plt.plot은 x, y 기입 필요x, 순서에 맞게
#sns.lineplot(y = hdcar_20['Close'], x = hdcar_20.index)# sns.lineplot은 x,y 기입 필요
plt.xlabel('Time')
plt.ylabel('Close_Price')
plt.title('HyunDai Car recent 20 years close_prices')

In [ ]:
# 5년 단위로 나눴을대
hdcar_0207 = fdr.DataReader('005380', '2002', '2007')
hdcar_0712 = fdr.DataReader('005380', '2007', '2012')
hdcar_1217 = fdr.DataReader('005380', '2012', '2017')
hdcar_1722 = fdr.DataReader('005380', '2017', '2022')
'''
#fig, ax = plt.subplots()
plt.figure(figsize = (20, 12))
plt.ylim([20000, 275000])

plt.subplot(2, 2, 1)
plt.plot(hdcar_0207.index.year, hdcar_0207['Close'])
plt.title('HD Car 2002 ~ 2007')
plt.xlabel('time')
plt.ylabel('Close price')

plt.subplot(2, 2, 2)
plt.plot(hdcar_0712.index.year, hdcar_0712['Close'])
plt.title('HD Car 2007 ~ 2012')
plt.xlabel('time')
plt.ylabel('Close price')


plt.subplot(2, 2, 3)
plt.plot(hdcar_1217.index.year, hdcar_1217['Close'])
plt.title('HD Car 2012 ~ 2017')
plt.xlabel('time')
plt.ylabel('Close price')


plt.subplot(2, 2, 4)
#plt.figure(figsize = (20, 12))
plt.plot(hdcar_1722.index.year, hdcar_1722['Close'])
plt.title('HD Car 2017 ~ 2022')
plt.xlabel('time')
plt.ylabel('Close price')
'''


In [ ]:

plt.figure(figsize = (16, 9))
plt.subplot(221)
plt.plot(hdcar_0207.index, hdcar_0207['Close'])
plt.xlabel('Time')
plt.ylabel('Close_Price')
plt.title('HyunDai Car 0207 close_prices')

#plt.figure(figsize = (16, 9))
plt.subplot(222)
plt.plot(hdcar_0712.index, hdcar_0712['Close'])
plt.xlabel('Time')
plt.ylabel('Close_Price')
plt.title('HyunDai Car 0712 close_prices')

#plt.figure(figsize = (16, 9))
plt.subplot(223)
plt.plot(hdcar_1217.index, hdcar_1217['Close'])
plt.xlabel('Time')
plt.ylabel('Close_Price')
plt.title('HyunDai Car 1217 close_prices')

#plt.figure(figsize = (16, 9))
plt.subplot(224)
plt.plot(hdcar_1722.index, hdcar_1722['Close'])
plt.xlabel('Time')
plt.ylabel('Close_Price')
plt.title('HyunDai Car 1722 close_prices')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaling_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
scaled_df = scaler.fit_transform(hdcar_20[scaling_cols])
scaled_df

In [ ]:
import pandas as pd

df = pd.DataFrame(scaled_df, columns = scaling_cols)
df

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop('Close', 1), df['Close'], test_size = 0.2, random_state = 0, shuffle = False)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
x_train

In [ ]:
# sequnce dataset
import tensorflow as tf

def window_dataset(series, window_size, batch_size, shuffle):
    serires = tf.expand_dims(series, axis = -1)
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size+1, shift = 1, drop_remainder = True)
    dataset = dataset.flat_map(lambda w : w.batch(window_size + 1))
    if shuffle:
        dataset = dataset.shuffle(1000)
    dataset = dataset.map(lambda w : (w[:-1], w[-1]))
    return dataset.batch(batch_size).prefetch(1)


In [ ]:
window_size = 20
batch_size = 32

train_data = window_dataset(y_train, window_size, batch_size, True)
test_data = window_dataset(y_test, window_size, batch_size, False)

for data in train_data.take(1):
    print('Data set X : batch_size = {0}, window_size = {1}, feature 개수 : {2}'.format(batch_size, window_size, data[0].shape))
    print('Data set Y : batch_size = {0}, window_size = {1}, feature 개수 : {2}'.format(batch_size, window_size, data[1].shape))

# Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential([
                    Conv1D(filters = 32, kernel_size = 5, padding = 'causal', activation = 'relu', input_shape = [window_size, 1]),
                    LSTM(16, activation = 'tanh'),
                    Dense(16, activation = 'relu'),
                    Dense(1),

])

loss = Huber()# Sequence 학습에 좋은 성능의 Huber
optimizer = Adam(0.001)
model.compile(loss = Huber(), optimizer = optimizer, metrics = ['mse'])

In [ ]:
# callback func
earlystopping = EarlyStopping(monitor = 'val_loss', patience = 10)
file_name = os.path.join('tmp', 'checkpointer.ckpt')
checkpoint = ModelCheckpoint(
    file_name, 
    save_weights_only = True, 
    save_best_only = True,
    monitor = 'val_loss',
    verbose = 1
    )

In [ ]:
history = model.fit(train_data, validation_data = (test_data), epochs = 50, callbacks = [checkpoint, earlystopping])

In [ ]:
model.load_weights(file_name)

In [ ]:
prediction = model.predict(test_data)
prediction.shape

In [ ]:
plt.figure(figsize = (12, 9))
plt.plot(np.asarray(y_test)[20:], label = 'actual')
plt.plot(prediction, label = 'prediction')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.optimizers import Adamax
model = Sequential([
                    Conv1D(filters = 32, kernel_size = 5, padding = 'causal', activation = 'relu', input_shape = [window_size, 1]),
                    LSTM(16, activation = 'tanh'),
                    Dense(16, activation = 'relu'),
                    Dense(1),

])

loss = Huber()# Sequence 학습에 좋은 성능의 Huber
optimizer = Adamax(0.0005)
model.compile(loss = Huber(), optimizer = optimizer, metrics = ['mse'])

earlystopping = EarlyStopping(monitor = 'val_loss', patience = 10)
file_name = os.path.join('tmp', 'checkpointer.ckpt')
checkpoint = ModelCheckpoint(
    file_name, 
    save_weights_only = True, 
    save_best_only = True,
    monitor = 'val_loss',
    verbose = 1
    )


history = model.fit(train_data, validation_data = (test_data), epochs = 50, callbacks = [checkpoint, earlystopping])

model.load_weights(file_name)

plt.figure(figsize = (12, 9))
plt.plot(np.asarray(y_test)[20:], label = 'actual')
plt.plot(prediction, label = 'prediction')
plt.legend()
plt.show()

In [ ]:
y_test

In [ ]:
hdcar_20

In [ ]:
hdcar_20.to_csv('Hyundai Car 20020102 ~ 20220119.csv', encoding = 'UTF8')

In [ ]:
hdcar_20[scaling_cols]

In [ ]:
hdcar_20[scaling_cols].to_csv('Hyundai Car 20020102 ~ 20220119 시가, 고가, 저가, 종가, 거래량.csv', encoding = 'UTF8')

In [ ]:
import time

def plot_plt(history, save_name, time):
    now = time
    plt.figure(figsize = (16, 8))
    
    plt.figure(figsize = (16, 8))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    
    plt.title('Train loss {0}'.format(save_name), fontsize = 20)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'val_loss'], loc = 'upper left')
    plt.savefig('./tmp/loss_graph/{0}_{1}_{2}_{3}_{4}_{5}_loss_graph.jpg'.format(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, save_name), bbox_inches = 'tight', dpi = 200)
    plt.show()
    plt.cla()

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
save_name = 'Stock_Prediction_HDcar'
now = time.localtime()
plot_plt(history, save_name, now)